In [ ]:
# Tensorflow to ONNX conversion is supported through the tf2onnx converter.
import onnxmltools
import tensorflow as tf
import tf2onnx

To convert a TensorFlow model, tf2onnx prefers a frozen TensorFlow graph with specified inputs and outputs for the graph, alongside a source checkpoint file.

```
python3 -m tf2onnx.convert 
    --input SOURCE_GRAPHDEF_PB
    --graphdef SOURCE_GRAPHDEF_PB
    --checkpoint SOURCE_CHECKPOINT
    --saved-model SOURCE_SAVED_MODEL
    [--inputs GRAPH_INPUTS]
    [--outputs GRAPH_OUTPUS]
    [--inputs-as-nchw inputs_provided_as_nchw]
    [--target TARGET]
    [--output TARGET_ONNX_GRAPH]
    [--target TARGET]
    [--continue_on_error]
    [--verbose]
    [--custom-ops list-of-custom-ops]
    [--opset OPSET]
    [--fold_const]
```

In [ ]:
!python3 -m tf2onnx.convert \
        --saved_model savedmodel \  # savedmodel directory
        --output model.onnx \       # output file name
        --target rs6 \
        --fold_const \
        --verbose

In [ ]:
!python3 -m tf2onnx.convert \
        --input model.pb \         # frozen graph input
        --inputs GRAPH_INPUTS \    # graph input names
        --outputs GRAPH_OUTPUTS \  # graph output names
        --output model.onnx \      # output file name
        --fold_const \
        --verbose

Alternatively, you could use a model from an active Tensorflow session in Python, freeze the graph, and then convert it, as demonstrated below.

In [ ]:
with tf.Session() as sess:
    # Note: this is a simple example Tensorflow model
    x = tf.placeholder(tf.float32, [2, 3], name="input")
    x_ = tf.add(x, x)
    _ = tf.identity(x_, name="output")
    onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph, input_names=["input:0"], output_names=["output:0"])
    onnx_model = onnx_graph.make_model(input_model_name)

In [ ]:
# Save as protobuf

onnxmltools.utils.save_model(onnx_model, output_onnx_model)